# Mandelbrot MPI Experiment Analysis

This notebook analyzes the performance and workload distribution of Mandelbrot MPI experiments.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import scienceplots

# Set scienceplots style
plt.style.use(['science', 'ieee'])
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 14,
    'axes.titlesize': 16,
    'xtick.labelsize': 12,
    'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# Set up paths
experiments_dir = Path('../experiments')
output_dir = Path('../Plots/analysis')
output_dir.mkdir(parents=True, exist_ok=True)

print("Setup complete!")

## Load Experiment Data

In [ ]:
def load_experiment_data(experiments_dir: Path):
    """Load all experiment CSV files."""
    data = {}
    
    for combo_dir in experiments_dir.glob("*_*"):  # static_blocking, etc.
        if not combo_dir.is_dir():
            continue
            
        combo = combo_dir.name
        csv_files = list(combo_dir.glob("*.csv"))
        
        if not csv_files:
            continue
            
        # Load and combine all CSV files for this combination
        dfs = []
        for csv_file in csv_files:
            df = pd.read_csv(csv_file)
            dfs.append(df)
        
        if dfs:
            combined_df = pd.concat(dfs, ignore_index=True)
            data[combo] = combined_df
            print(f"Loaded {len(csv_files)} experiments for {combo}: {len(combined_df)} total rows")
    
    return data

# Load data
data = load_experiment_data(experiments_dir)
print(f"\nLoaded {len(data)} experiment combinations")

## Data Overview

In [ ]:
# Show overview of loaded data
for combo, df in data.items():
    print(f"\n{combo.upper()}:")
    print(f"  Experiments: {df['experiment_id'].nunique()}")
    print(f"  Total rows: {len(df)}")
    print(f"  Processes tested: {sorted(df['num_processes'].unique())}")
    print(f"  Image sizes: {df[['image_width', 'image_height']].drop_duplicates().values.tolist()}")
    
    # Show sample experiment
    sample_exp = df[df['experiment_id'] == df['experiment_id'].iloc[0]]
    print(f"  Sample experiment: {sample_exp['experiment_id'].iloc[0]}")
    print(f"    Processes: {sample_exp['num_processes'].iloc[0]}")
    print(f"    Wall clock time: {sample_exp['wall_clock_time'].iloc[0]:.4f}s")
    print(f"    Load balance: {sample_exp['load_balance_score'].iloc[0]:.4f}")

## Performance Analysis

In [ ]:
# Performance comparison: Wall clock time vs processes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Wall clock time vs processes
for combo, df in data.items():
    if df.empty:
        continue
        
    # Group by number of processes and get mean wall clock time
    perf_data = df.groupby('num_processes')['wall_clock_time'].mean().reset_index()
    ax1.plot(perf_data['num_processes'], perf_data['wall_clock_time'], 
            marker='o', linewidth=2, markersize=8, label=combo.replace('_', ' ').title())

ax1.set_xlabel('Number of MPI Processes')
ax1.set_ylabel('Wall Clock Time (seconds)')
ax1.set_title('Performance: Wall Clock Time')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Speedup vs processes
for combo, df in data.items():
    if df.empty:
        continue
        
    # Calculate speedup (time_1proc / time_nproc)
    perf_data = df.groupby('num_processes')['wall_clock_time'].mean().reset_index()
    if len(perf_data) > 1:
        single_proc_time = perf_data[perf_data['num_processes'] == 1]['wall_clock_time'].iloc[0]
        speedup = single_proc_time / perf_data['wall_clock_time']
        ax2.plot(perf_data['num_processes'], speedup, 
                marker='s', linewidth=2, markersize=8, label=combo.replace('_', ' ').title())

ax2.set_xlabel('Number of MPI Processes')
ax2.set_ylabel('Speedup')
ax2.set_title('Scaling: Speedup')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'performance_analysis.pdf', bbox_inches='tight')
plt.show()

## Load Balance Analysis

In [ ]:
# Load balance analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Load balance score vs processes
for combo, df in data.items():
    if df.empty:
        continue
        
    lb_data = df.groupby('num_processes')['load_balance_score'].mean().reset_index()
    ax1.plot(lb_data['num_processes'], lb_data['load_balance_score'], 
            marker='o', linewidth=2, markersize=8, label=combo.replace('_', ' ').title())

ax1.set_xlabel('Number of MPI Processes')
ax1.set_ylabel('Load Balance Score')
ax1.set_title('Load Balancing Quality')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 1.1)

# Plot 2: Time breakdown (computation vs communication)
for combo, df in data.items():
    if df.empty:
        continue
        
    # Group by number of processes and get mean times
    time_data = df.groupby('num_processes').agg({
        'computation_time': 'mean',
        'communication_time': 'mean'
    }).reset_index()
    
    ax2.plot(time_data['num_processes'], time_data['computation_time'], 
            marker='o', linewidth=2, markersize=8, label=f'{combo.replace("_", " ").title()} (Comp)')
    ax2.plot(time_data['num_processes'], time_data['communication_time'], 
            marker='s', linestyle='--', linewidth=2, markersize=8, 
            label=f'{combo.replace("_", " ").title()} (Comm)')

ax2.set_xlabel('Number of MPI Processes')
ax2.set_ylabel('Time (seconds)')
ax2.set_title('Time Breakdown: Computation vs Communication')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'load_balance_analysis.pdf', bbox_inches='tight')
plt.show()

## Individual Experiment Analysis

Let's analyze each experiment in detail:

In [ ]:
# Function to analyze a single experiment
def analyze_experiment(exp_data, exp_id, combo):
    """Analyze a single experiment in detail."""
    
    # Get experiment metadata
    exp_info = exp_data.iloc[0]
    num_processes = exp_info['num_processes']
    image_size = f"{exp_info['image_width']}x{exp_info['image_height']}"
    wall_clock_time = exp_info['wall_clock_time']
    
    print(f"\n{'='*60}")
    print(f"EXPERIMENT: {combo.upper()}")
    print(f"ID: {exp_id}")
    print(f"Image Size: {image_size}")
    print(f"Processes: {num_processes}")
    print(f"Wall Clock Time: {wall_clock_time:.4f}s")
    print(f"Load Balance Score: {exp_info['load_balance_score']:.4f}")
    print(f"{'='*60}")
    
    # Create comprehensive analysis
    fig = plt.figure(figsize=(20, 12))
    
    # 1. Time distribution pie chart
    ax1 = plt.subplot(2, 3, 1)
    comp_time = exp_info['computation_time']
    comm_time = exp_info['communication_time']
    other_time = wall_clock_time - comp_time - comm_time
    
    times = [comp_time, comm_time, other_time]
    labels = ['Computation', 'Communication', 'Other']
    colors = ['#ff9999', '#66b3ff', '#99ff99']
    
    wedges, texts, autotexts = ax1.pie(times, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
    ax1.set_title(f'Time Distribution\n{combo.replace("_", " ").title()}')
    
    # 2. Chunks per rank
    ax2 = plt.subplot(2, 3, 2)
    if num_processes > 1:
        rank_data = exp_data[exp_data['rank'] > 0]  # Exclude master
        if not rank_data.empty:
            ranks = rank_data['rank'].values
            chunks = rank_data['chunks_processed'].values
            
            bars = ax2.bar(ranks, chunks, alpha=0.7, color='skyblue')
            ax2.set_xlabel('MPI Rank')
            ax2.set_ylabel('Chunks Processed')
            ax2.set_title('Chunks per Rank')
            ax2.grid(True, alpha=0.3)
            
            # Add value labels
            for bar, chunk_count in zip(bars, chunks):
                height = bar.get_height()
                ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                       f'{chunk_count}', ha='center', va='bottom')
            
            # Add statistics
            mean_chunks = np.mean(chunks)
            std_chunks = np.std(chunks)
            ax2.axhline(y=mean_chunks, color='red', linestyle='--', 
                       label=f'Mean: {mean_chunks:.1f}')
            ax2.axhline(y=mean_chunks + std_chunks, color='orange', linestyle=':', 
                       label=f'$\\pm 1\\sigma$: {std_chunks:.1f}')
            ax2.legend()
        else:
            ax2.text(0.5, 0.5, 'No worker data', ha='center', va='center', transform=ax2.transAxes)
            ax2.set_title('Chunks per Rank')
    else:
        ax2.text(0.5, 0.5, 'Single process', ha='center', va='center', transform=ax2.transAxes)
        ax2.set_title('Chunks per Rank')
    
    # 3. Rows per rank (if we have row data)
    ax3 = plt.subplot(2, 3, 3)
    if num_processes > 1 and 'row_ranges' in exp_data.columns:
        rank_data = exp_data[exp_data['rank'] > 0]
        if not rank_data.empty and not rank_data['row_ranges'].isna().all():
            # Parse row ranges and calculate total rows per rank
            rows_per_rank = []
            for _, row in rank_data.iterrows():
                row_ranges = row['row_ranges']
                if pd.notna(row_ranges) and row_ranges:
                    # Parse ranges like "0-24, 25-49"
                    total_rows = 0
                    for range_str in str(row_ranges).split(', '):
                        if '-' in range_str:
                            start, end = map(int, range_str.split('-'))
                            total_rows += end - start + 1
                    rows_per_rank.append(total_rows)
                else:
                    rows_per_rank.append(0)
            
            if rows_per_rank:
                ranks = rank_data['rank'].values
                bars = ax3.bar(ranks, rows_per_rank, alpha=0.7, color='lightgreen')
                ax3.set_xlabel('MPI Rank')
                ax3.set_ylabel('Rows Processed')
                ax3.set_title('Rows per Rank')
                ax3.grid(True, alpha=0.3)
                
                # Add value labels
                for bar, row_count in zip(bars, rows_per_rank):
                    height = bar.get_height()
                    ax3.text(bar.get_x() + bar.get_width()/2., height + 1,
                           f'{row_count}', ha='center', va='bottom')
            else:
                ax3.text(0.5, 0.5, 'No row data', ha='center', va='center', transform=ax3.transAxes)
                ax3.set_title('Rows per Rank')
        else:
            ax3.text(0.5, 0.5, 'No row data', ha='center', va='center', transform=ax3.transAxes)
            ax3.set_title('Rows per Rank')
    else:
        ax3.text(0.5, 0.5, 'Single process', ha='center', va='center', transform=ax3.transAxes)
        ax3.set_title('Rows per Rank')
    
    # 4. Computation time per rank
    ax4 = plt.subplot(2, 3, 4)
    if num_processes > 1:
        rank_data = exp_data[exp_data['rank'] > 0]
        if not rank_data.empty:
            comp_times = rank_data['computation_time'].values
            bars = ax4.bar(ranks, comp_times, alpha=0.7, color='lightcoral')
            ax4.set_xlabel('MPI Rank')
            ax4.set_ylabel('Computation Time (s)')
            ax4.set_title('Computation Time per Rank')
            ax4.grid(True, alpha=0.3)
            
            # Add value labels
            for bar, time in zip(bars, comp_times):
                height = bar.get_height()
                ax4.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                       f'{time:.3f}', ha='center', va='bottom')
        else:
            ax4.text(0.5, 0.5, 'No worker data', ha='center', va='center', transform=ax4.transAxes)
            ax4.set_title('Computation Time per Rank')
    else:
        ax4.text(0.5, 0.5, 'Single process', ha='center', va='center', transform=ax4.transAxes)
        ax4.set_title('Computation Time per Rank')
    
    # 5. Communication time per rank
    ax5 = plt.subplot(2, 3, 5)
    if num_processes > 1:
        rank_data = exp_data[exp_data['rank'] > 0]
        if not rank_data.empty:
            comm_times = rank_data['communication_time'].values
            bars = ax5.bar(ranks, comm_times, alpha=0.7, color='lightblue')
            ax5.set_xlabel('MPI Rank')
            ax5.set_ylabel('Communication Time (s)')
            ax5.set_title('Communication Time per Rank')
            ax5.grid(True, alpha=0.3)
            
            # Add value labels
            for bar, time in zip(bars, comm_times):
                height = bar.get_height()
                ax5.text(bar.get_x() + bar.get_width()/2., height + 0.001,
                       f'{time:.3f}', ha='center', va='bottom')
        else:
            ax5.text(0.5, 0.5, 'No worker data', ha='center', va='center', transform=ax5.transAxes)
            ax5.set_title('Communication Time per Rank')
    else:
        ax5.text(0.5, 0.5, 'Single process', ha='center', va='center', transform=ax5.transAxes)
        ax5.set_title('Communication Time per Rank')
    
    # 6. Load balance metrics
    ax6 = plt.subplot(2, 3, 6)
    ax6.axis('off')
    ax6.set_title('Load Balance Metrics', fontsize=14, fontweight='bold')
    
    if num_processes > 1:
        rank_data = exp_data[exp_data['rank'] > 0]
        if not rank_data.empty:
            chunks = rank_data['chunks_processed'].values
            mean_chunks = np.mean(chunks)
            max_chunks = np.max(chunks)
            min_chunks = np.min(chunks)
            load_balance = min_chunks / max_chunks if max_chunks > 0 else 1.0
            cv = np.std(chunks)/mean_chunks if mean_chunks > 0 else 0.0
            
            metrics = [
                ('Load Balance Score', f"{load_balance:.4f}"),
                ('Mean Chunks/Rank', f"{mean_chunks:.1f}"),
                ('Max Chunks/Rank', f"{max_chunks}"),
                ('Min Chunks/Rank', f"{min_chunks}"),
                ('Std Dev', f"{np.std(chunks):.2f}"),
                ('Coefficient of Variation', f"{cv:.3f}"),
            ]
        else:
            metrics = [('No worker data available', '')]
    else:
        metrics = [('Single process execution', '')]
    
    for i, (metric, value) in enumerate(metrics):
        ax6.text(0.1, 0.9 - i*0.12, f"{metric}: {value}", 
                fontsize=12, transform=ax6.transAxes)
    
    plt.suptitle(f'Detailed Analysis: {combo.replace("_", " ").title()}\n{exp_id}', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    
    # Save individual experiment analysis
    safe_exp_id = exp_id.replace('/', '_').replace(':', '_')
    plt.savefig(output_dir / f'experiment_{combo}_{safe_exp_id}.pdf', bbox_inches='tight')
    plt.show()
    
    return exp_info

# Analyze each experiment
experiment_summaries = {}
for combo, df in data.items():
    experiment_ids = df['experiment_id'].unique()
    
    for exp_id in experiment_ids:
        exp_data = df[df['experiment_id'] == exp_id]
        summary = analyze_experiment(exp_data, exp_id, combo)
        experiment_summaries[exp_id] = summary

## Summary Statistics

In [ ]:
# Create summary table
summary_data = []
for exp_id, summary in experiment_summaries.items():
    summary_data.append({
        'Experiment ID': exp_id,
        'Configuration': summary['schedule'] + '_' + summary['communication'],
        'Processes': summary['num_processes'],
        'Image Size': f"{summary['image_width']}x{summary['image_height']}",
        'Wall Clock Time (s)': f"{summary['wall_clock_time']:.4f}",
        'Computation Time (s)': f"{summary['computation_time']:.4f}",
        'Communication Time (s)': f"{summary['communication_time']:.4f}",
        'Load Balance Score': f"{summary['load_balance_score']:.4f}",
        'Total Chunks': summary['total_chunks']
    })

summary_df = pd.DataFrame(summary_data)
print("\nEXPERIMENT SUMMARY:")
print("="*80)
print(summary_df.to_string(index=False))

# Save summary to CSV
summary_df.to_csv(output_dir / 'experiment_summary.csv', index=False)
print(f"\nSummary saved to {output_dir / 'experiment_summary.csv'}")

## Scaling Analysis

In [ ]:
# Calculate and plot scaling efficiency
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot 1: Speedup vs processes
for combo, df in data.items():
    if df.empty:
        continue
        
    perf_data = df.groupby('num_processes')['wall_clock_time'].mean().reset_index()
    if len(perf_data) > 1:
        single_proc_time = perf_data[perf_data['num_processes'] == 1]['wall_clock_time'].iloc[0]
        speedup = single_proc_time / perf_data['wall_clock_time']
        efficiency = (speedup / perf_data['num_processes']) * 100
        
        ax1.plot(perf_data['num_processes'], speedup, 
                marker='o', linewidth=2, markersize=8, 
                label=f'{combo.replace("_", " ").title()} (Speedup)')
        ax2.plot(perf_data['num_processes'], efficiency, 
                marker='s', linewidth=2, markersize=8, 
                label=f'{combo.replace("_", " ").title()} (Efficiency)')

# Add ideal speedup line
if len(perf_data) > 1:
    processes = perf_data['num_processes'].values
    ax1.plot(processes, processes, 'k--', alpha=0.5, label='Ideal Speedup')

ax1.set_xlabel('Number of MPI Processes')
ax1.set_ylabel('Speedup')
ax1.set_title('Scaling: Speedup')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Number of MPI Processes')
ax2.set_ylabel('Efficiency (%)')
ax2.set_title('Scaling: Parallel Efficiency')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 105)

plt.tight_layout()
plt.savefig(output_dir / 'scaling_analysis.pdf', bbox_inches='tight')
plt.show()

## Workload Distribution Analysis

In [ ]:
# Analyze workload distribution patterns
print("\nWORKLOAD DISTRIBUTION ANALYSIS:")
print("="*50)

for combo, df in data.items():
    print(f"\n{combo.upper()}:")
    
    # Get multi-process experiments
    multi_proc_df = df[df['num_processes'] > 1]
    if multi_proc_df.empty:
        print("  No multi-process experiments")
        continue
    
    for _, exp in multi_proc_df.iterrows():
        exp_id = exp['experiment_id']
        num_proc = exp['num_processes']
        
        # Get worker data for this experiment
        exp_data = df[df['experiment_id'] == exp_id]
        workers = exp_data[exp_data['rank'] > 0]
        
        if not workers.empty:
            chunks = workers['chunks_processed'].values
            mean_chunks = np.mean(chunks)
            std_chunks = np.std(chunks)
            cv = std_chunks / mean_chunks if mean_chunks > 0 else 0
            
            print(f"  {exp_id} ({num_proc} processes):")
            print(f"    Chunks per rank: {chunks.tolist()}")
            print(f"    Mean: {mean_chunks:.1f}, Std: {std_chunks:.1f}, CV: {cv:.3f}")
            print(f"    Load balance: {exp['load_balance_score']:.4f}")
            
            # Show row ranges if available
            if 'row_ranges' in workers.columns and not workers['row_ranges'].isna().all():
                print(f"    Row ranges:")
                for _, worker in workers.iterrows():
                    if pd.notna(worker['row_ranges']):
                        print(f"      Rank {worker['rank']}: {worker['row_ranges']}")
                    else:
                        print(f"      Rank {worker['rank']}: No row data")

print("\nAnalysis complete!")
print(f"All plots saved to: {output_dir}")